# Efficiency: locality; caching
_COSC 208, Introduction to Computer Systems, 2023-11-13_

## Announcements
* Project 3 due today @ 11pm

## Outline
* Warm-up
* Temporal vs. spatial locality
* CPU caches
* Cache replacement

## Warm-up

* Q1: _For each of the following characteristics, circle the type(s) of memory to which the characteristic applies. (HDD = Hard Disk Drive; RAM = Random Access Memory; SSD = Solid State Drive)_

| Characteristic | | | | | |
|-----|-|-|-|-|-|
| <br/>Cheapest<br/><br/> | Cache | HDD | RAM | Registers | SSD |
| <br/>Fastest<br/><br/> | Cache | HDD | RAM | Registers | SSD |
| <br/>On CPU<br/><br/> | Cache | HDD | RAM | Registers | SSD |
| <br/>Volatile<br/><br/> | Cache | HDD | RAM | Registers | SSD |
| <br/>Size measured in megabytes (MB)<br/>in a present day laptop | Cache | HDD | RAM | Registers | SSD |
| <br/>Size measured in gigabytes (GB)<br/>in a present day laptop | Cache | HDD | RAM | Registers | SSD |
| <br/>Size measured in terabytes (TB)<br/>in a present day laptop | Cache | HDD | RAM | Registers | SSD |

    Cheapest: HDD
    Fastest: Registers
    On CPU: Cache, Registers
    Volatile: Cache, RAM, Registers
    Size in MB: Cache
    Size in GB: RAM
    Size in TB: HDD, SSD

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Data movement

* Recall: _How does data move between the CPU, main memory, and secondary storage in the von Neumann Architecture?_ — bus
* _Why does data move between registers and main memory?_ — not enough room in registers to store all values used by a program at runtime
* _How can we move less data (i.e., perform fewer loads and stores)?_
* Better use of registers — Loads and stores are unnecessary when the value of a register is not changed between store and load instructions involving the same register and memory address
    * Example load which is _unnecessary_
        ```
        str x0, [sp,#4]
        ldr x0, [sp,#4] // Can eliminiate
        ```
    * Example load store which is _necessary_
        ```
        str w0, [sp,#4]
        mov w0, #0x1
        str w0, [sp]
        ldr w0, [sp,#4]
        ```
    * Better register assignments to eliminate loads (and stores)
        ```
        str w0, [sp,#4]
        mov w1, #0x1
        str w1, [sp]
        ldr w0, [sp,#4] // Can eliminiate
        ```
    * Must preserve calling conventions
        * Parameters are stored in w/x0, w/x1, ...
        * Return value is stored in w/x0
        * Caller must store register values into caller's stack frame before `bl` to callee — actually only needed if values in registers are needed by caller after `bl` and callee overwrites the values in those registers
* Leverage locality
    * Add additional memory to the CPU — i.e., a cache
    * Optimize code to improve locality

## Temporal vs. spatial locality

* _What is temporal locality?_
    * Access the same data repeatedly
    * E.g., for loop variable
* _What is spatial locality?_
    * Access data with a similar scope
    * E.g., next item in array
    * E.g., local variables/parameters, which are stored in the same stack frame
* Analogies for temporal and spatial locality
    * Book storage (Dive Into Systems Section 11.3.2)
        * Temporal locality — store most frequently used books at your desk, less frequently used books on your bookshelf, and least frequently used books at the library
        * Spatial locality — checkout books on the same/nearby subjects when you go to the library
    * Groceries (pre-class questions 3 & 4)
        * Temporal locality — you store food you eat frequently in the front of the refrigerator, while you store food you eat infrequently in the back of the refrigerator
        * Spatial locality — you organize the items on your grocery list based on the aisle in which they are located
    * _With a partner, develop your own analogy for temporal and spatial locality_

## CPU caches

* _Why do we have caches on the CPU?_ — accessing main memory is ~100x slower than accessing a register
* Store instructions and data (stack, heap, etc.) from main memory
* Three levels — L1, L2, and L3
* Range in size from a few KB to a few MB – L1 is smallest and fastest; L3 is largest and slowest
* Cache line (i.e., cache entry) is typically larger than a word — e.g., 128 bytes
    * _Why?_ — spatial locality
* What happens when we write to memory?
    * Write through cache — write to the cache and main memory
    * Write back cache — initially write to the cache; write to main memory when the entry is evicted from the cache
    * What are the advantages of each approach?
        * Write through cache ensures consistency between CPU cores
        * Write back cache only incurs the overhead of accessing main memory when absolutely necessary

## Cache replacement

* If a cache is full, then a cache entry must be removed so different data can be placed in the cache
* Cache replacement policy governs which data is removed
* _What should a good cache replacement policy do?_ — maximize the number of cache hits (or minimize the number of cache misses)
    * Evaluation metric: Hit ratio = number of hits / total number of memory accesses
* _How do we determine which cache entry to replace?_
* Optimal replacement policy – replace the entry that will be accessed furthest in the future
    * Impractical because we don’t know data access patterns a priori
* Least Recently Used (LRU)
    * LRU assumes a item that was accessed recently will be accessed again soon – temporal locality
    * Downside: lots of overhead to implement — need to store an ordered list of items and move an item up in the list whenever it’s accessed
    * Where does this go wrong? — when working-set size (i.e., number of repeatedly accessed entries) is (slightly) greater than size of the cache
* First-in First-out (FIFO)
    * Simple to implement
    * Doesn’t consider the importance of a cache entry
* Random
    * Even simpler to implement
    * Doesn’t consider the importance of a cache entry

* Assume a cache can hold 3 entries and the following 15 data accesses occur: 
```
3, 4, 4, 5, 3, 2, 3, 4, 1, 4, 4, 2, 5, 2, 4
```
* Q2: _What is the sequence of hits, insertions, and replacements that occur when an **optimal** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -5/+2, H3, H4, -3/+1, H4, H4, H2, -1/+5, H2, H4
Hit ratio = 9/15 = 60%
```

<p style="height:6em;"></p>

* Q3: _What is the sequence of hits, insertions, and replacements that occur when a **first in first out (FIFO)** cache replacement algorithm is used?_

    Postponed to next class

<p style="height:6em;"></p>

* Q4: _What is the sequence of hits, insertions, and replacements that occur when a **least recently used (LRU)** cache replacement algorithm is used?_

    Postponed to next class

<p style="height:6em;"></p>

## Extra practice

* Q5: _What output is produced by this program, assuming the starting process's PID is 7034 and PIDs are sequentially assigned to new processes._

In [4]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int result = fork();
    if (result == 0) {
        printf("A %d %d\n", result, getpid());
    } else if (result > 0) {
        wait(NULL);
        printf("B %d %d\n", result, getpid());
    }
    int result2 = fork();
    if (result2 == 0) {
        printf("C %d %d\n", result2, getpid());
    } else if (result > 0) {
        wait(NULL);
        printf("D %d %d\n", result2, getpid());
    }
}

Begin
A 0 7035
B 7035 7034
C 0 7036
C 0 7037
D 7037 7034


<p style="height:6em;"></p>